In [1]:
from __future__ import print_function
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import sys
import os
import argparse
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.autograd import Variable
from data import *
from PIL import Image
from utils.augmentations import SSDAugmentation
from layers.functions import Detect


import torch.utils.data as data
from ssd import build_ssd

/home/broiron/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def test_net(save_folder, net, cuda, testset, transform, thresh):
    # dump predictions and assoc. ground truth to text file for now
    filename = os.path.join(save_folder, 'test1.txt')
    num_images = len(testset)
    for i in range(num_images):
        print('Testing image {:d}/{:d}....'.format(i+1, num_images))
        img, annotation, _, _, img_id = testset.pull_item(i)
        print(img_id)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        x = img.unsqueeze(0)
        x = x.type(torch.FloatTensor)
        x = x.to(device)
        x = Variable(x)

        with open(filename, mode='a') as f:
            f.write('\nGROUND TRUTH FOR: '+img_id+'\n')
            for box in annotation:
                f.write('label: '+' || '.join(str(b) for b in box)+'\n')

        net = net.to(device)

        y = net(x)      # forward pass
        detections = y.data
        # scale each detection back up to the image
        scale = torch.Tensor([img.shape[1], img.shape[0],
                             img.shape[1], img.shape[0]])
        pred_num = 0
        for i in range(detections.size(1)):
            j = 0
            while detections[0, i, j, 0] >= args.visual_threshold:
                if pred_num == 0:
                    with open(filename, mode='a') as f:
                        f.write('PREDICTIONS: '+'\n')
                score = detections[0, i, j, 0]
                label_name = CUSTOM_CLASSES[i-1]
                pt = (detections[0, i, j, 1:]*scale).cpu().numpy()
                coords = (pt[0], pt[1], pt[2], pt[3])
                pred_num += 1
                with open(filename, mode='a') as f:
                    f.write(str(pred_num)+' label: '+label_name+' score: ' +
                            str(score) + ' '+' || '.join(str(c) for c in coords) + '\n')
                print(str(pred_num)+' label: '+label_name+' score: ' +
                            str(score) + ' '+' || '.join(str(c) for c in coords))
                j += 1


In [2]:
detect = Detect(num_classes=2, bkg_label=1, top_k=200, conf_thresh=0.01, nms_thresh=0.445)

In [3]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
dataset = COCODetection(root='/home/broiron/broiron/line_dataset_vol1_coco/', 
                        transform=SSDAugmentation(300, MEANS), image_set='test')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [20]:
img, an, _, _ = dataset.pull_item(10)

/home/broiron/broiron/model_train/ssd-pytorch-custom/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


In [5]:
print(len(dataset))

100


In [6]:
# setting with default value
# optimizer = optim.SGD(net.parameters(), lr=1e-5, momentum=0.9, # 1e-3 -> 1e-5
#                         weight_decay=5e-4) 
'''
criterion = MultiBoxLoss(num_classes=2, overlap_thresh=0.5, prior_for_matching=True, 
                         bkg_label=1, neg_mining=True, neg_pos=3, neg_overlap=0.5, 
                         encode_target=False, use_gpu=True)
'''

criterion = MultiBoxLoss(2, 0.5, True, 0, True, 3, 0.5, False, True)

In [7]:
def detection_collate(batch):
    """Custom collate fn for dealing with batches of images that have a different
    number of associated object annotations (bounding boxes).
    Arguments:
        batch: (tuple) A tuple of tensor images and lists of annotations
    Return:
        A tuple containing:
            1) (tensor) batch of images stacked on their 0 dim
            2) (list of tensors) annotations for a given image are stacked on 0 dim
    """
    targets = []
    imgs = []
    for sample in batch:
        imgs.append(sample[0])
        targets.append(torch.FloatTensor(sample[1]))
    return torch.stack(imgs, 0), targets

In [8]:
net = build_ssd(phase='test', size=300, num_classes=2)
net.load_state_dict(torch.load('./weights/linedataset_vol1_1a.pth'))
net.eval()
print("Finished loading model!")

loc_loss = 0
conf_loss = 0
epoch = 0
batch_size = 1

print('loading dataset...')

print('Testing SSD on: ', dataset.name)

step_index = 0

data_loader = data.DataLoader(dataset, batch_size, num_workers=4, shuffle=False, collate_fn=detection_collate,
                             pin_memory=True)
print('Data loader length...', len(data_loader))

Finished loading model!
loading dataset...
Testing SSD on:  MS COCO
Data loader length... 100


In [10]:
batch_iterator = iter(data_loader)
for iteration in range(0, len(data_loader)): # 1개 sample 순회
    try:
        images, targets = next(batch_iterator)
    except:
        print('error occured in iterator')
    with torch.no_grad():
        images = Variable(images.to(device))
        targets = [Variable(ann.to(device)) for ann in targets]
        out = net.forward(images) # forward pass   
        out = detect.forward(out[0], out[1], out[2])
        
    detections = out.data
    scale = torch.Tensor(im)
        
    
    

/home/broiron/broiron/model_train/ssd-pytorch-custom/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
/home/broiron/broiron/model_train/ssd-pytorch-custom/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
/home/broiron/broiron/model_train/ssd-pytorch-custom/utils/augmentations.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with

torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
torch.Size([1, 8732, 2])
error occured in iterator